In [1]:
import lime
import sklearn
import numpy as np
import sklearn
import sklearn.ensemble
import sklearn.metrics

import scipy
import json
import pickle
import pandas as pd
import tuner_library
import utilities as util
import matplotlib.pyplot as plt

In [2]:
def getRmse1(actual_preds, target_preds):
    rmse_val = 0
    for x in range(len(actual_preds)):
        rmse_val += (float(target_preds[x])- actual_preds[x])*(float(target_preds[x]) - actual_preds[x])
    #print(rmse_val)
    rmse_val = float(rmse_val)/len(target_preds)
    return rmse_val

In [3]:
#ask user to input the name of classes and store it in classNames as a json
class_names = ['negative', 'positive']
class_names = json.dumps(class_names)

#converting json to array object which will be used by the code further
class_names = json.loads(class_names)
print(class_names)

#get the text from the user in the string form(not json)


['negative', 'positive']


In [4]:
df = pd.read_csv('https://github.com/clairett/pytorch-sentiment-classification/raw/master/data/SST2/train.tsv', delimiter='\t', header=None)


In [5]:
from sklearn.model_selection import train_test_split

train_features, test_features, train_labels, test_labels = train_test_split(df[0], df[1])

In [6]:
import sklearn.feature_extraction
vectorizer = sklearn.feature_extraction.text.TfidfVectorizer(lowercase=False)
train_vectors = vectorizer.fit_transform(train_features)
test_vectors = vectorizer.transform(test_features)

In [7]:
rf = sklearn.ensemble.RandomForestClassifier(n_estimators=500)
rf.fit(train_vectors, train_labels)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [8]:
pred = rf.predict(test_vectors)
sklearn.metrics.f1_score(test_labels, pred, average='binary')
getRmse1(test_labels.tolist(), pred.tolist())

0.2751445086705202

In [9]:
from lime import lime_text
from sklearn.pipeline import make_pipeline
c = make_pipeline(vectorizer, rf)

In [10]:
import pickle
pickle.dump(c, open("model.pkl", 'wb'))

In [ ]:
i = 200
text = df[0][i]
text

In [ ]:
from tuner_library import tuner
tl = tuner(text,'model.pkl', class_names)
result = util.JsonToArray(tl.get_Prediction()) 
result

In [ ]:
#ask user to input the value of sigma and store it in sigma as a json
sigma = 25
sigma = json.dumps(sigma)

#converting json to array object which will be used by the code further
sigma = json.loads(sigma)
sigma

In [ ]:
#get the explanation for a particular sigma (all the explanation fields are filled)
exp = tl.get_Explanation(sigma)
print(exp.rf_score)
exp.save_to_file('explanation_25.html')

In [ ]:
k = 0.39062499968
tl.sigmas = k*2**np.arange(13)

In [ ]:
for sigma in tl.sigmas:
    exp = tl.get_Explanation(sigma)    

In [ ]:
#for plotting entropy v/s sigma curve
sigmas, entropies = tl.get_Sigma_Entropy()
#get the rmse values(all the explanation models) for all the sigmas 
sigmas,constant_rmse, linear_rmse, decisionTree_rmse, randomForest_rmse = tl.get_RMSEs_And_Sigmas()

In [ ]:
#plot the sigma v/s rmses 
axes=plt.gca()

plt.xscale('log', basex = 2)
plt.xlabel("sigma")
plt.ylabel("rmse");

plt.plot(sigmas, randomForest_rmse,'-o',label = 'random_forest')
plt.plot(sigmas, decisionTree_rmse,'-o',label = 'decision_tree')
plt.plot(sigmas, linear_rmse,'-o', label = 'linear_model')
# plt.plot(width, constant_rmse,'-o',label = 'constant_model')
# plt.suptitle('Example id = ' + str(idx))

legend = plt.legend(loc='upper left', shadow=True)

legend.get_frame()

#plot the sigma v/s rmses 
# axes=plt.gca()

# plt.xscale('log', basex = 2)
# plt.xlabel("sigma")
# plt.ylabel("rmse");

# plt.plot(sigmas[0:4], randomForest_rmse[0:4],'-o',label = 'random_forest')
# plt.plot(sigmas[0:4], decisionTree_rmse[0:4],'-o',label = 'decision_tree')
# plt.plot(sigmas[0:4], linear_rmse[0:4],'-o', label = 'linear_model')
# plt.plot(sigmas[0:4], constant_rmse[0:4],'-o',label = 'constant_model')
# # plt.suptitle('Example id = ' + str(idx))

# legend = plt.legend(loc='upper left', shadow=True)

# legend.get_frame()

In [ ]:
#for plotting weight distribution of neighbourhood points for a particular sigma
weights, predictions = tl.get_Weight_Distribution_Plot_Per_Sigma(25)

#plot the weight distribution of neighbour points for a particular sigma
axes=plt.gca()
plt.ylabel("weights")
plt.xlabel("prediction");
plt.ylim(-0.5, 1.5)
major_ticks = np.arange(-0.5, 1.5, 0.5)
axes.set_yticks(major_ticks) 
plt.xlim(0,1)
print(sigma)
plt.plot(predictions, weights,'o')

In [ ]:

print(tl.getRange_Constant_Model())
print(tl.getRange_Linear_Model())

In [ ]:
print(linear_rmse)
print(constant_rmse)